### - 1 in every 8 people in the world live with a mental disorder
### - The majority of suicides worldwide are related to mental health disorders; depression, substance use, and psychosis
### - The World Health Organization (WHO) reports that suicide is one of the leading causes of death among individuals aged 15-19

## The Idea:

## Show ingishts about what people suffering from those disorders are talking about through topic modelling

## Build a mental health recommendation system offering personalized advice derived from shared community experiences of like-minded people by leveraging text analytics, NLP and LLM.

## Recommender System

In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/mental-health-codes"
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/mental-health-codes')
#import utilities as ut

%reload_ext autoreload
%autoreload 2

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/mental-health-codes


In [2]:
#Importing necessary libraries
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import WordNetLemmatizer
import torch
from transformers import BertTokenizer, BertModel
from transformers import pipeline
import io

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


#### We are scraping data from a medical forum for 5 mental disorders people are talking about. ####

#### We have the tag(disorder), title of the concern, the user concerns, and the suggestions offered by people for that concern ####

In [3]:
import requests
import re
import urllib.request
from urllib.parse import urlparse
import unicodedata
from urllib.parse import urljoin
import logging
from lxml.html import fromstring

# Insights about disorders based on Topic Modelling

In [4]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.0 MB/s eta 0:00:00


In [5]:
from bertopic import BERTopic

df=pd.read_csv('counsel_chat.csv')
df.head(5)

,Unnamed: 0,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,split
0,0,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,"If everyone thinks you're worthless, then mayb...",1,2899,train
1,1,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Robin Landwehr, DBH, LPCC, NCCMental Health in...",https://counselchat.com/therapists/robin-landw...,"Hello, and thank you for your question and see...",1,3514,train
2,2,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Lee KingI use an integrative approach to treat...,https://counselchat.com/therapists/lee-king,First thing I'd suggest is getting the sleep y...,0,5,train
3,3,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Shauntai Davis-YearginPersonalized, private on...",https://counselchat.com/therapists/shauntai-da...,Therapy is essential for those that are feelin...,0,31,train
4,4,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Jordan WhiteLicensed Social Worker at Oak Root...,https://counselchat.com/therapists/jordan-white,I first want to let you know that you are not ...,0,620,train


In [6]:
df['topic'].unique()

array(['depression', 'anxiety', 'parenting', 'self-esteem',
       'relationship-dissolution', 'workplace-relationships',
       'spirituality', 'trauma', 'domestic-violence', 'anger-management',
       'sleep-improvement', 'intimacy', 'grief-and-loss',
       'substance-abuse', 'family-conflict', 'marriage',
       'eating-disorders', 'relationships', 'lgbtq', 'behavioral-change',
       'addiction', 'legal-regulatory', 'professional-ethics', 'stress',
       'human-sexuality', 'social-relationships', 'children-adolescents',
       'military-issues', 'self-harm', 'diagnosis',
       'counseling-fundamentals'], dtype=object)

In [7]:
df.drop_duplicates(subset='questionText', keep='first', inplace=True)
df[:5]

,Unnamed: 0,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,split
0,0,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,"If everyone thinks you're worthless, then mayb...",1,2899,train
23,23,1,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Noorayne ChevalierMA, RP, CCC, CCAC, LLP (Mich...",https://counselchat.com/therapists/noorayne-ch...,Let me start by saying there are never too man...,2,477,train
70,70,2,How do I find out the cause of my depression a...,I have been feeling more and more down for ove...,https://counselchat.com/questions/how-do-i-fin...,depression,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,Answers about our inner lives are most success...,0,41,train
72,72,3,How do I overcome my anxierty and depression?,I’m facing severe depression and anxiety and I...,https://counselchat.com/questions/how-do-i-ove...,depression,Dr. Melissa ValentinaHonor Yourself and Live Y...,https://counselchat.com/therapists/dr-melissa-...,Have you used meditation or hypnosis? Relaxing...,0,26,train
81,81,4,Why am I upset or down every day even when not...,How can I get to a place where I can be conten...,https://counselchat.com/questions/why-am-i-ups...,depression,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,Your question is a fascinating one!As humans w...,0,37,train


In [8]:
import nltk
nltk.download('punkt_tab')
def remove_stopwords(text):
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [word for word in filtered_words if word.isalnum()]
    return " ".join(filtered_words)

df['questionText'] = df['questionText'].astype(str).apply(remove_stopwords)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [9]:
df_type_0= df[(df['topic']=="parenting") | (df['topic']=="family-conflict")].reset_index(drop=True)
df_type_1= df[df['topic']=="anxiety"].reset_index(drop=True)
df_type_2= df[df['topic']=="depression"].reset_index(drop=True)
df_type_3= df[(df['topic']=="social-relationships") | (df['topic']=="relationships")].reset_index(drop=True)

In [14]:
docs = df_type_1['questionText'].to_list()
model = BERTopic(verbose=True)
topics, probabilities = model.fit_transform(docs)
model.get_topic_freq().head(11)

2024-12-25 06:30:38,140 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2024-12-25 06:30:43,137 - BERTopic - Embedding - Completed ✓
2024-12-25 06:30:43,144 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-12-25 06:30:43,402 - BERTopic - Dimensionality - Completed ✓
2024-12-25 06:30:43,410 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-25 06:30:43,425 - BERTopic - Cluster - Completed ✓
2024-12-25 06:30:43,437 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-12-25 06:30:43,467 - BERTopic - Representation - Completed ✓


,Topic,Count
0,-1,39
1,0,37
2,1,12


In [15]:
model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,39,-1_like_feel_get_thoughts,"[like, feel, get, thoughts, people, know, scar...",[go school feel like everyone judging even fri...
1,0,37,0_feel_anxiety_years_like,"[feel, anxiety, years, like, relationship, tim...",[dad refuses acknowledge anxieties going parti...
2,1,12,1_people_talk_want_know,"[people, talk, want, know, meet, popular, grad...",[feel scared meet people know even relatives e...


In [16]:
model.visualize_barchart()

In [18]:
docs = df_type_1['questionText'].to_list()
model = BERTopic(verbose=True)
topics, probabilities = model.fit_transform(docs)
model.get_topic_freq().head(11)

2024-12-25 06:30:55,270 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2024-12-25 06:30:57,818 - BERTopic - Embedding - Completed ✓
2024-12-25 06:30:57,819 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-12-25 06:30:57,977 - BERTopic - Dimensionality - Completed ✓
2024-12-25 06:30:57,980 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-25 06:30:57,996 - BERTopic - Cluster - Completed ✓
2024-12-25 06:30:58,002 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-12-25 06:30:58,024 - BERTopic - Representation - Completed ✓


,Topic,Count
1,-1,39
2,0,23
0,1,16
3,2,10


In [20]:
model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,39,-1_like_feel_get_think,"[like, feel, get, think, know, people, thought...",[go school feel like everyone judging even fri...
1,0,23,0_anxiety_feel_help_attacks,"[anxiety, feel, help, attacks, around, ca, get...",[mg Xanax twice day past month helping take 1 ...
2,1,16,1_people_know_want_fear,"[people, know, want, fear, talk, feel, meet, a...",[socially awkward always want popular opportun...
3,2,10,2_years_feel_anxiety_relationship,"[years, feel, anxiety, relationship, someone, ...",[feel insecure life think wife truly wants rel...


In [21]:
model.visualize_barchart()

In [22]:
docs = df_type_2['questionText'].to_list()
model = BERTopic(verbose=True)
topics, probabilities = model.fit_transform(docs)
model.get_topic_freq().head(11)


2024-12-25 06:31:01,636 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

2024-12-25 06:31:07,168 - BERTopic - Embedding - Completed ✓
2024-12-25 06:31:07,173 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-12-25 06:31:07,411 - BERTopic - Dimensionality - Completed ✓
2024-12-25 06:31:07,413 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-25 06:31:07,424 - BERTopic - Cluster - Completed ✓
2024-12-25 06:31:07,429 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-12-25 06:31:07,448 - BERTopic - Representation - Completed ✓


,Topic,Count
1,0,47
2,-1,45
0,1,40


In [23]:
model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,45,-1_feel_know_depression_like,"[feel, know, depression, like, never, friends,...",[depressed quite trying work boyfriend rock ho...
1,0,47,0_depression_anxiety_get_time,"[depression, anxiety, get, time, help, feel, d...",[ongoing problem life young adult right drivin...
2,1,40,1_feel_like_want_happy,"[feel, like, want, happy, life, years, time, k...",[almost never happy Half time feel anything fi...


In [24]:
model.visualize_barchart()

In [25]:
docs = df_type_3['questionText'].to_list()
model = BERTopic(verbose=True)
topics, probabilities = model.fit_transform(docs)
model.get_topic_freq().head(11)

2024-12-25 06:31:07,850 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-12-25 06:31:12,504 - BERTopic - Embedding - Completed ✓
2024-12-25 06:31:12,507 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-12-25 06:31:12,693 - BERTopic - Dimensionality - Completed ✓
2024-12-25 06:31:12,697 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-25 06:31:12,706 - BERTopic - Cluster - Completed ✓
2024-12-25 06:31:12,712 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-12-25 06:31:12,731 - BERTopic - Representation - Completed ✓


,Topic,Count
2,0,42
1,-1,37
0,1,22
3,2,11


In [26]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,37,-1_dog_want_boyfriend_get,"[dog, want, boyfriend, get, girlfriend, know, ...",[losing feelings know love much sometimes thin...
1,0,42,0_like_together_told_get,"[like, together, told, get, love, want, know, ...",[early 20 something virgin boyfriend early 20s...
2,1,22,1_friends_friend_guy_really,"[friends, friend, guy, really, want, relations...",[hanging close guy friend started kissing afte...
3,2,11,2_people_like_talking_messages,"[people, like, talking, messages, issues, wo, ...",[know someone extremely sensitive stress sensi...


In [27]:
model.visualize_barchart()

# Recommender System

In [28]:
#We can take in a user input in this form

# disorder_types = input("anxiety", "depression", "relationships/social-realtionships")

disorder_level= "relationships/social-realtionships"

##Now taking a self-esteem incident as input to see the possible nearest counseling answers

In [29]:
#This too should be an user input

user_rant="Everyone around me is much smarter and flaunts it. Everyone around me is skinny, and here I am trying to throw up so I'm not fat. Everything I do is wrong, and I can't seem to do anything right! No one else at school seems to feel the way I do! Is this normal teenage girls feelings? I don't think these feelings are normal."

In [30]:
df=pd.read_csv('counsel_chat.csv')
df.head()

,Unnamed: 0,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,split
0,0,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,"If everyone thinks you're worthless, then mayb...",1,2899,train
1,1,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Robin Landwehr, DBH, LPCC, NCCMental Health in...",https://counselchat.com/therapists/robin-landw...,"Hello, and thank you for your question and see...",1,3514,train
2,2,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Lee KingI use an integrative approach to treat...,https://counselchat.com/therapists/lee-king,First thing I'd suggest is getting the sleep y...,0,5,train
3,3,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Shauntai Davis-YearginPersonalized, private on...",https://counselchat.com/therapists/shauntai-da...,Therapy is essential for those that are feelin...,0,31,train
4,4,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Jordan WhiteLicensed Social Worker at Oak Root...,https://counselchat.com/therapists/jordan-white,I first want to let you know that you are not ...,0,620,train


In [31]:
def remove_stopwords(text):
    words = nltk.word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [word for word in filtered_words if word.isalnum()]
    return " ".join(filtered_words)

# Assuming df['Concern'] contains your text data
df['questionText'] = df['questionText'].str.lower()
print(df['questionText'])

0       i'm going through some things with my feelings...
1       i'm going through some things with my feelings...
2       i'm going through some things with my feelings...
3       i'm going through some things with my feelings...
4       i'm going through some things with my feelings...
                              ...                        
2124    after first meeting the client, what is the pr...
2125    after first meeting the client, what is the pr...
2126    after first meeting the client, what is the pr...
2127    after first meeting the client, what is the pr...
2128    after first meeting the client, what is the pr...
Name: questionText, Length: 2129, dtype: object


In [32]:
df_disorder=df[(df['topic']=="social-relationships") | (df['topic']=="relationships")]

In [33]:
df_disorder= df_disorder[['questionText']].drop_duplicates().reset_index(drop=True)
df_disorder['User Concern']=user_rant

In [34]:
df_disorder

,questionText,User Concern
0,there's this boy who asked me out awhile ago. ...,Everyone around me is much smarter and flaunts...
1,i have suffered many things at home and school...,Everyone around me is much smarter and flaunts...
2,i have a friend that who i used to be in a rel...,Everyone around me is much smarter and flaunts...
3,i've been with my partner for 4 years. she's g...,Everyone around me is much smarter and flaunts...
4,i’ve been in a long distance relationship with...,Everyone around me is much smarter and flaunts...
...,...,...
107,i'm having problems shutting them out and putt...,Everyone around me is much smarter and flaunts...
108,"i'm having a quinceañera, and the girls don't ...",Everyone around me is much smarter and flaunts...
109,i would like to be able to have more positive ...,Everyone around me is much smarter and flaunts...
110,in this argument my friends are all against me...,Everyone around me is much smarter and flaunts...


In [35]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.5 MB/s eta 0:00:00


In [36]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Calculate the sentiment score for the single "User Concern" value
user_concern_sentiment = analyzer.polarity_scores(df_disorder.iloc[0]['User Concern'])['compound']

# Apply the sentiment score to all rows in the "Concern" column

df_disorder['Concern_Sentiment'] = df_disorder['questionText'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
df_disorder['User_Concern_Sentiment'] = user_concern_sentiment

# Print the updated DataFrame
df_disorder

,questionText,User Concern,Concern_Sentiment,User_Concern_Sentiment
0,there's this boy who asked me out awhile ago. ...,Everyone around me is much smarter and flaunts...,0.4215,-0.4374
1,i have suffered many things at home and school...,Everyone around me is much smarter and flaunts...,0.6757,-0.4374
2,i have a friend that who i used to be in a rel...,Everyone around me is much smarter and flaunts...,0.5423,-0.4374
3,i've been with my partner for 4 years. she's g...,Everyone around me is much smarter and flaunts...,-0.7852,-0.4374
4,i’ve been in a long distance relationship with...,Everyone around me is much smarter and flaunts...,0.0516,-0.4374
...,...,...,...,...
107,i'm having problems shutting them out and putt...,Everyone around me is much smarter and flaunts...,-0.4019,-0.4374
108,"i'm having a quinceañera, and the girls don't ...",Everyone around me is much smarter and flaunts...,-0.2755,-0.4374
109,i would like to be able to have more positive ...,Everyone around me is much smarter and flaunts...,0.7501,-0.4374
110,in this argument my friends are all against me...,Everyone around me is much smarter and flaunts...,-0.4950,-0.4374


In [37]:
# We assume that a sentiment score of greater than 0.05 (based on Vader) of the user rant means the user is talking about something cheerful

# In other words, we need to keep the user rant if its negative or neutral
# Same for the concerns in our data

df_disorder= df_disorder[(df_disorder['Concern_Sentiment']<0.05) & (df_disorder['User_Concern_Sentiment']<0.05)]

In [38]:
#Applying word vector based similarity and finding score of all Scraped concerns with the user input
import spacy.cli
spacy.cli.download("en_core_web_md")

nlp = spacy.load('en_core_web_md')
def word_vector_similarity(text1, text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    return doc1.similarity(doc2)


# this adds the simlarity score of each comment in the concern column with the user input
df_disorder['Similarity Score'] = df_disorder['questionText'].apply(word_vector_similarity, text2=user_rant)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [39]:
#Similarity Score Threshold setting
df_disorder=df_disorder[df_disorder['Similarity Score']>0.5].sort_values(by='Similarity Score', ascending=False).reset_index(drop=True)[:10]

In [40]:
df_sug= df_disorder.merge(df, on='questionText', how='left')[['questionText','User Concern','Similarity Score','Concern_Sentiment','User_Concern_Sentiment','answerText']]
df_sug['answerText']= df_sug['answerText'].astype(str)

# Combine rows into one paragraph
paragraph = ' '.join(df_sug['answerText'])

# Load a pre-trained spaCy model (English)
nlp = spacy.load('en_core_web_md')
nlp.max_length = 2000000 # or even higher

# Split the paragraph into sentences
doc = nlp(paragraph)
sentences = [sent.text for sent in doc.sents]

In [45]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Initialize a matrix to store sentence similarity scores
num_sentences = len(sentences)
similarity_matrix = np.zeros((num_sentences, num_sentences))

# Calculate sentence similarity using cosine similarity
for i in range(num_sentences):
    for j in range(i, num_sentences):  # Only calculate each pair once
        if i != j:
            vector1 = nlp(sentences[i]).vector
            vector2 = nlp(sentences[j]).vector
            similarity = cosine_similarity([vector1], [vector2])[0][0]
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity  # Symmetric

# Print the sentence similarity matrix (optional)
print("Sentence Similarity Matrix:")
print(similarity_matrix)

Sentence Similarity Matrix:
[[0.         0.7956754  0.65767699 ... 0.80301154 0.79799777 0.80703348]
 [0.7956754  0.         0.63878059 ... 0.70139134 0.69013941 0.68960178]
 [0.65767699 0.63878059 0.         ... 0.67664164 0.67427045 0.66197568]
 ...
 [0.80301154 0.70139134 0.67664164 ... 0.         0.9106769  0.95190281]
 [0.79799777 0.69013941 0.67427045 ... 0.9106769  0.         0.94508868]
 [0.80703348 0.68960178 0.66197568 ... 0.95190281 0.94508868 0.        ]]


In [46]:
# Sum up the similarity values for each sentence
sentence_similarity_scores = np.sum(similarity_matrix, axis=1)

# Create a list of (sentence, score) tuples
sentence_scores_tuples = [(sentence, score) for sentence, score in zip(sentences, sentence_similarity_scores)]

# Sort the tuples by score in descending order
sentence_scores_tuples_sorted = sorted(sentence_scores_tuples, key=lambda x: x[1], reverse=True)

# Specify the desired percentage, e.g., 10%,  Calculate the threshold score based on the desired percentage (e.g., 10%)
percentage = 10

num_sentences = len(sentences)
threshold_score = np.percentile(sentence_similarity_scores, 100 - percentage)

# Create a list of sentences that have scores greater than or equal to the threshold
top_10_sentences = [sentence for sentence, score in sentence_scores_tuples if score >= threshold_score]

# Print the top 10% of sentences
for i, sentence in enumerate(top_10_sentences, 1):
    print(f"Top {i}: {sentence}")

Top 1: Once you take the journey to really understand who you are and unconditionally loving all that is you, there will never be an alone moment. 
Top 2: Instead, its when you follow up by inquiring and listening to others that you  may discover they feel and think and struggle just like you do. 
Top 3: To help you ask for the right type of support, you can ask yourself what it is about the situation that is bothering you and how you would wish for your friends and family to respond to you.
Top 4: Maybe for right now, until you are able to find in person friends whom you're able to feel hear you the way you'd like to be heard, find online forums and groups of likeminded people.
Top 5: How did you find out that the girls aren't happy with their dresses?Did you hear this directly from one of them or is this third hand information that someone else told you about one of the girls in your party?The answer matters because gossiping instead of directly telling you about a problem, is a prob

In [47]:
# Function to filter sentences with non-negative sentiment
def filter_positive_sentences(sentences):
    positive_sentences = []
    for sentence in sentences:
        sentiment = analyzer.polarity_scores(sentence)
        if sentiment['compound'] >= 0:
            positive_sentences.append(sentence)
    return positive_sentences

# Filter the list of sentences
top_10_sentences = filter_positive_sentences(top_10_sentences)

# Print the updated list of positive sentences
for i, sentence in enumerate(top_10_sentences, 1):
    print(f"Positive {i}: {sentence}")

Positive 1: Once you take the journey to really understand who you are and unconditionally loving all that is you, there will never be an alone moment. 
Positive 2: Instead, its when you follow up by inquiring and listening to others that you  may discover they feel and think and struggle just like you do. 
Positive 3: To help you ask for the right type of support, you can ask yourself what it is about the situation that is bothering you and how you would wish for your friends and family to respond to you.
Positive 4: Maybe for right now, until you are able to find in person friends whom you're able to feel hear you the way you'd like to be heard, find online forums and groups of likeminded people.
Positive 5: If you think about the wording of a question which would motivate you to respond, then this formula will very likely be the same for many others who read your post.
Positive 6: If you can phrase it in such a way that they recognize that you are asking for information and not blam

In [48]:
#Just limiting the number of sentences to summarize for summarization speed without GPU
if len(top_10_sentences)>25:
    top_10_sentences=top_10_sentences[:25]
top_10_sentences

['Once you take the journey to really understand who you are and unconditionally loving all that is you, there will never be an alone moment.\xa0',
 'Instead, its when\xa0you follow up by inquiring and listening to others that\xa0you \xa0may discover they feel and think and struggle just like you do.\xa0',
 'To help you ask for the right type of support, you can ask yourself what it is about the situation that is bothering you and how you would wish for your friends and family to respond to you.',
 "Maybe for right now, until you are able to find in person friends whom you're able to feel hear you the way you'd like to be heard, find online forums and groups of likeminded people.",
 'If you think about the wording of a question which would motivate you to respond, then this formula will very likely be the same for many others who read your post.',
 'If you can phrase it in such a way that they recognize that you are asking for information and not blaming them for not answering you, tha

In [60]:
# Using BART tuned for text-summarization through transformers pipelines

from transformers import pipeline

# Create the Summarization Pipeline
summarizer = pipeline("summarization")

# Input text to be summarized
text = ' '.join(top_10_sentences)

# Generate a summary
summary = summarizer(text, max_length=63, min_length=30, do_sample=False)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [61]:
# Print the summary
print("Here is a summarized form of what people are sugeesting to people with similar concerns:\n\n",summary[0]['summary_text'])

Here is a summarized form of what people are sugeesting to people with similar concerns:

  Ask yourself what it is about the situation that is bothering you and how you would wish for your friends and family to respond to you . One way to check if you're totally misreading him, is to examine whether you feel similarly in other relationships . If your girlfriend can truly see that you view her
